In [ ]:
from models.vgg import VGG16
import os
import torch
import torch.nn as nn
from torch.optim import SGD, lr_scheduler
from torchvision import datasets,transforms
from torch.utils.data import DataLoader
from tqdm import tqdm

device = torch.device("cuda")
root = "./datasets"
download = os.path.exists("./datasets/cifar-10-batches-py")

model = VGG16()
model.to(device)

transform_train = transforms.Compose([
    transforms.ToTensor(),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomCrop(size=32, padding=4)
])

transform_test = transforms.ToTensor()

trainset = datasets.CIFAR10(root="./datasets", download=False, train=True, transform=transform_train)
testset = datasets.CIFAR10(root="./datasets", download=False, train=False, transform=transform_test)
trainloader = DataLoader(trainset, shuffle=True, batch_size=128)
testloader = DataLoader(testset, shuffle=False, batch_size=256)

loss_fn = nn.CrossEntropyLoss()
optimizer = SGD(model.parameters(),lr=0.1,momentum=0.9,weight_decay=5e-4,nesterov=True)
scheduler = lr_scheduler.StepLR(optimizer,step_size=100,gamma=0.1)

epochs = 200
best_val_acc = 0
for e in range(epochs):
    with tqdm(trainloader,desc=f"{e+1}/{epochs} epochs") as t:
        running_correct = 0
        running_loss = 0
        running_total = 0
        model.train()
        for i, (x,y) in enumerate(t):
            out = model(x.to(device))
            loss = loss_fn(out,y.to(device))
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            running_correct += (out.max(dim=1)[1]==y.to(device)).sum().item()
            running_loss += loss.item()*x.size(0)
            running_total += x.size(0)
            if i < len(trainloader)-1:
                t.set_postfix({
                    "train_acc": running_correct/running_total,
                    "train_loss": running_loss/running_total
                })
            else:
                scheduler.step()
                model.eval()
                val_correct = 0
                val_loss = 0
                val_total = 0
                with torch.no_grad():
                    for _, (x,y) in zip(range(4),testloader):
                        out = model(x.to(device))
                        loss = loss_fn(out,y.to(device))
                        val_correct += (out.max(dim=1)[1]==y.to(device)).sum().item()
                        val_loss += loss.item()*x.size(0)
                        val_total += x.size(0)
                        t.set_postfix({
                            "train_acc": running_correct/running_total,
                            "train_loss": running_loss/running_total,
                            "val_acc": val_correct/val_total,
                            "val_loss": val_loss/val_total
                        })
                    if val_correct/val_total > best_val_acc:
                        best_val_acc = val_correct/val_total
                        torch.save(model.state_dict(), "./model_weights/cifar_vgg_clean.pt")

124/200 epochs: 100%|██████████| 391/391 [00:56<00:00,  6.93it/s, train_acc=0.975, train_loss=0.0764, val_acc=0.915, val_loss=0.308]


197/200 epochs:  65%|██████▌   | 256/391 [00:31<00:12, 11.04it/s, train_acc=0.978, train_loss=0.0719]